In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

In [92]:
search = "music" #hashtag

URL = f"https://ads.tiktok.com/business/creativecenter/inspiration/popular/{search}/pc/en"
page = requests.get(URL)

soup = BeautifulSoup(page.text)
scripts = soup.find("script", id='__NEXT_DATA__')

In [99]:
find_content = soup.find('script', id="__NEXT_DATA__")
str_content = str(find_content).split('<')[1].split('>')[1]

In [103]:
json_content = json.loads(str_content)['props']['pageProps']
json_content['data']

{'pagination': {'page': 1, 'size': 3, 'total': 100, 'hasMore': True},
 'soundList': [{'author': 'Dj Kampung',
   'cover': 'https://p16-sg.tiktokcdn.com/aweme/100x100/tos-alisg-v-2774/61951971962245328cc03da59c763e2f.jpeg',
   'link': 'https://www.tiktok.com/music/x-7193959214111442946',
   'promoted': False,
   'rank': 1,
   'title': 'Big Boy Speed Up',
   'trend': [{'time': 1676419200, 'value': 0.35},
    {'time': 1676505600, 'value': 0.46},
    {'time': 1676592000, 'value': 0.51},
    {'time': 1676678400, 'value': 0.56},
    {'time': 1676764800, 'value': 0.79},
    {'time': 1676851200, 'value': 0.98},
    {'time': 1676937600, 'value': 1}],
   'clipId': '7193959214111442946',
   'countryCode': 'US',
   'ifCml': False,
   'isSearch': False,
   'onListTimes': None,
   'rankDiff': 1,
   'rankDiffType': 1,
   'songId': '7193959213800884225',
   'urlTitle': 'Big-Boy-Speed-Up'},
  {'author': 'PinkPantheress & Ice Spice',
   'cover': 'https://p16-sg.tiktokcdn.com/aweme/100x100/tos-alisg-v-27

In [135]:
search = "hashtag" #music
hashtag = "jhope"
period = "120"
country = "FR"

URL = f"https://ads.tiktok.com/business/creativecenter/{search}/{hashtag}/pc/en?countryCode={country}&period={period}"
page = requests.get(URL)

soup = BeautifulSoup(page.text)
stats = [e.text for e in soup.find_all('span', {'class': 'title--gvWft title--eM6Wz'})]
trend = soup.find('span', {'class': 'bannerDesc--CORuD bannerDesc--McarQ'}).text

In [136]:
stats

['14K', '11M', '67M', '65B']

In [142]:
trend = soup.find('span', {'class': 'bannerDesc--CORuD bannerDesc--McarQ'}).text

In [123]:
soup

[<html lang="en"><head><link href="/business/creativecenter/api/sitemap" rel="sitemap" title="Sitemap" type="application/xml"/><meta content="on" http-equiv="x-dns-prefetch-control"/><link as="font" href="https://sf16-scmcdn-sg.ibytedtos.com/obj/static-sg/ads/creative_hub_i18n/fonts/sofiapro-bold-webfont.woff2" rel="preload" type="font/woff2"/><link as="font" href="https://sf16-scmcdn-sg.ibytedtos.com/obj/static-sg/ads/creative_hub_i18n/fonts/sofiapro-medium-webfont.woff2" rel="preload" type="font/woff2"/><link as="font" href="https://sf16-scmcdn-sg.ibytedtos.com/obj/static-sg/ads/creative_hub_i18n/fonts/sofiapro-regular-webfont.woff2" rel="preload" type="font/woff2"/><link href="//sf16-scmcdn-sg.ibytedtos.com" rel="preconnect"/><link href="//p16-cc-sg.ibyteimg.com" rel="preconnect"/><link href="//p16-sign-sg.tiktokcdn.com" rel="preconnect"/><link href="//sf16-scmcdn-sg.ibytedtos.com" rel="dns-prefetch"/><link href="//p16-cc-sg.ibyteimg.com" rel="dns-prefetch"/><link href="//p16-sign-s